# **Blog Outline and Content Generation Using LangGraph and OpenAI**

**Prompt chaining is a technique in AI workflows where the output from one language model prompt is used as input for the next prompt, creating a sequence or "chain" of prompts.** This allows more complex tasks to be broken down into smaller, manageable steps, with each step building on the results of the previous one. In this code, the first node generates a blog outline based on the title, and the second node uses that outline to produce detailed blog content showing a clear chain of prompts feeding into each other to build a complete output.

In [1]:
# Import required components from LangGraph and LangChain-OpenAI
from langgraph.graph import StateGraph, START, END  # Workflow graph and entry/exit points
from langchain_openai import ChatOpenAI             # LLM wrapper for OpenAI models
from typing import TypedDict                        # For strong type annotation of state objects
from dotenv import load_dotenv                      # To read OpenAI (or other) keys from a .env file

# Load environment variables (OPENAI_API_KEY) from a .env file.
load_dotenv()

True

In [2]:
# Instantiate the OpenAI language model. Here, we specify the model as "gpt-5".
model = ChatOpenAI(model="gpt-5")

In [3]:
# Define the structure of the state for the blog generation workflow.
# This includes the blog title, the generated outline, and the final blog content.
class BlogState(TypedDict):
    title: str      # The blog topic/title provided as input
    outline: str    # The generated outline for the blog (to be created in one node)
    content: str    # The detailed blog content (to be created using the outline)

In [4]:
# Node 1: Generate a detailed blog outline based on the blog title.
def create_outline(state: BlogState) -> BlogState:
    title = state['title']   # Extract the title from the state

    # Create the prompt for the language model to generate an outline
    prompt = f"Generate a detailed outline for a blog on a Topic - {title}"

    # Call the language model to get the outline content
    outline = model.invoke(prompt).content

    # Update the state with the generated outline
    state['outline'] = outline

    # Return the updated state
    return state

In [5]:
# Node 2: Generate detailed blog content using the title and the previously generated outline.
def create_blog(state: BlogState) -> BlogState:
    title = state['title']       # Get the blog title
    outline = state['outline']   # Get the outline created in the previous node

    # Create a prompt to generate blog content incorporating the outline
    prompt = f"Generate a detailed blog on a title - {title} using the following outline - \n{outline}"

    # Call the language model to generate the blog content
    content = model.invoke(prompt).content

    # Store the generated blog content back into the state
    state['content'] = content

    # Return the updated state
    return state

In [6]:
# Initialize a StateGraph with the BlogState structure.
graph = StateGraph(BlogState)

# Add the nodes (processing steps) to the graph.
graph.add_node('create_outline', create_outline)   # Node to generate the blog outline
graph.add_node('create_blog', create_blog)         # Node to generate the blog content

# Define the execution order: start → create_outline → create_blog → end
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

# Compile the graph into an executable workflow.
workflow = graph.compile()

In [7]:
# Initial input state with the blog title.
initial_state = {'title': 'Future of Agentic AI'}

# Run the workflow with the initial state.
final_state = workflow.invoke(initial_state)

In [9]:
print(final_state['title'])

Future of Agentic AI


In [10]:
print(final_state['outline'])

Below is a comprehensive, publication-ready outline you can hand to a writer or use as a blueprint for a long-form blog (2,500–3,500 words) on the Future of Agentic AI.

Working title options
- The Future of Agentic AI: From Helpful Tools to Autonomous Collaborators
- Agentic AI 2030: Capabilities, Risks, and the Path to Responsible Autonomy
- Beyond Chatbots: How Agentic AI Will Reshape Work, Software, and Society

Audience and angle
- Audience: Product leaders, engineers, researchers, policymakers, enterprise executives, and tech-savvy readers.
- Angle: Balanced, forward-looking view that combines technical architecture, real-world applications, governance, and practical adoption guidance.

Thesis
- Agentic AI marks a shift from passive assistants to proactive, tool-using systems capable of planning, taking actions, and collaborating with humans and other agents. Over the next decade, it will transform software, operations, and knowledge work—if we pair capability growth with robust 

In [11]:
print(final_state['content'])

The Future of Agentic AI: From Helpful Tools to Autonomous Collaborators

1) Hook and context (intro)
At 8:30 a.m., your enterprise agent has already negotiated a delivery window with a vendor, rebooked your team’s stand-up to avoid a product review conflict, filed five expense reports with correct cost centers, and pushed a performance dashboard refresh to the CFO—complete with a one-paragraph variance analysis and links to the underlying data. Nobody wrote a script for any of this. You told the agent your goals, guardrails, and preferences; it planned, acted through APIs and services, and kept you informed without micromanagement.

That is the promise of agentic AI: systems that perceive, plan, decide, and act through tools and services with varying levels of autonomy. Unlike passive assistants that only respond to prompts, agentic systems can interpret goals, decompose tasks, use tools, collaborate with people and other agents, and adapt to feedback and context.

Why now? Because se